# Estimación del modelo

In [5]:
import pandas as pd
import numpy as np

La estimación de los modelos se realiza a partir de https://bashtage.github.io/linearmodels/panel/examples/examples.html

In [23]:
bd = pd.read_csv('base_final.csv')

In [24]:
year = pd.Categorical(bd["ao"])

In [25]:
data = bd.set_index(["edo","ao"])

In [26]:
data['ao'] = year

In [27]:
print(data.head())

                     clave_ent  homicidio  robocasa  robocalle  robocarro  \
edo            ao                                                           
Aguascalientes 2015          1        211      2525       1754       1995   
               2016          1        233      2459       1353       3110   
               2017          1        258      3327       1949       3624   
               2018          1        283      3355       2216       3309   
               2019          1        269      2817       1780       2399   

                     extorsion  tasa_hom  tasa_rcasa  tasa_rcalle  \
edo            ao                                                   
Aguascalientes 2015         39     15.84      189.59       131.70   
               2016         70     17.19      181.43        99.83   
               2017        121     18.75      241.83       141.66   
               2018        138     20.28      240.36       158.76   
               2019        116     19.00      

In [28]:
data["logpibe"] = np.log(data["pibe"])
data["prop25a34"] = data["edad25a34"]*100/data["pob_tot"]

In [29]:
import linearmodels as linearmodels
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

## Estimación PooledOLS

In [30]:
exog_vars = ['prop25a34','desocupa','logpibe','pobreza']
exog = sm.add_constant(data[exog_vars])
mod = PooledOLS(data.tasa_rcasa, exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:             tasa_rcasa   R-squared:                        0.5040
Estimator:                  PooledOLS   R-squared (Between):              0.5469
No. Observations:                 128   R-squared (Within):              -0.0335
Date:                Sun, Jul 19 2020   R-squared (Overall):              0.5040
Time:                        23:50:42   Log-likelihood                   -712.74
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      31.251
Entities:                          32   P-value                           0.0000
Avg Obs:                       4.0000   Distribution:                   F(4,123)
Min Obs:                       4.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             31.251
                            

/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/core/fromnumeric.py:42: RuntimeWarning: invalid value encountered in reduce
  result = getattr(asarray(obj), method)(*args, **kwds)
/srv/conda/envs/notebook/lib/python3.7/site-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


## Estimación con Efectos Aleatorios

In [31]:
exog_vars = ['prop25a34','desocupa','logpibe','pobreza']
exog = sm.add_constant(data[exog_vars])
mod = PooledOLS(data.tasa_rcasa, exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:             tasa_rcasa   R-squared:                        0.5040
Estimator:                  PooledOLS   R-squared (Between):              0.5469
No. Observations:                 128   R-squared (Within):              -0.0335
Date:                Sun, Jul 19 2020   R-squared (Overall):              0.5040
Time:                        23:50:52   Log-likelihood                   -712.74
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      31.251
Entities:                          32   P-value                           0.0000
Avg Obs:                       4.0000   Distribution:                   F(4,123)
Min Obs:                       4.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             31.251
                            

In [32]:
from linearmodels.panel import RandomEffects
mod = RandomEffects(data.tasa_rcasa, exog)
re_res = mod.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:             tasa_rcasa   R-squared:                        0.2133
Estimator:              RandomEffects   R-squared (Between):              0.5319
No. Observations:                 128   R-squared (Within):               0.0108
Date:                Sun, Jul 19 2020   R-squared (Overall):              0.4935
Time:                        23:51:00   Log-likelihood                   -606.99
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      8.3355
Entities:                          32   P-value                           0.0000
Avg Obs:                       4.0000   Distribution:                   F(4,123)
Min Obs:                       4.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             8.3355
                            

## Comparación efectos aleatorios vs PoolOLS

In [19]:
from linearmodels.panel import compare
print(compare({'RE':re_res,'Pooled':pooled_res}))

                 Model Comparison                 
                                   RE       Pooled
--------------------------------------------------
Dep. Variable              tasa_rcasa   tasa_rcasa
Estimator               RandomEffects    PooledOLS
No. Observations                  128          128
Cov. Est.                  Unadjusted   Unadjusted
R-squared                      0.2133       0.5040
R-Squared (Within)             0.0108      -0.0335
R-Squared (Between)            0.5319       0.5469
R-Squared (Overall)            0.4935       0.5040
F-statistic                    8.3355       31.251
P-value (F-stat)               0.0000       0.0000
===================== =============== ============
const                          148.36       121.34
                             (0.4666)     (0.6842)
prop25a34                      37.524       40.646
                             (2.5189)     (4.7447)
desocupa                       12.535       5.2567
                             (2